### Segmenting and Clustering Neighborhoods in Toronto

In [350]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# !pip install geopy
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!pip install folium
import folium

In [351]:
#get table
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [352]:
page = urllib.request.urlopen(url)

In [353]:
data = BeautifulSoup(page, "lxml")

In [354]:
all_tables=data.find_all("table")
all_tables

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighborhood
 </th></tr>
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>
 <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>
 <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>
 <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>
 <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue, Humber Valley Village
 </td></tr>
 <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>
 <tr>
 <td>M2B


In [355]:
right_table=data.find('table', class_='wikitable sortable')
right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td>

In [356]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [357]:
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C

df.head(5)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [358]:
#clean data
indexnames = df[df['Borough'].str.contains('Not assigned')].index
df.drop(indexnames, inplace = True)

In [363]:
df = df.reset_index(drop=True)
df = df.replace('\n','', regex=True)
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [360]:
print(df['Neighborhood'].unique())
print(df['Borough'].unique())
#after delete rows with a borough that is Not assigned, there is no row with
#a neighbor that is not assigned

['Parkwoods\n' 'Victoria Village\n' 'Regent Park, Harbourfront\n'
 'Lawrence Manor, Lawrence Heights\n'
 "Queen's Park, Ontario Provincial Government\n"
 'Islington Avenue, Humber Valley Village\n' 'Malvern, Rouge\n'
 'Don Mills\n' 'Parkview Hill, Woodbine Gardens\n'
 'Garden District, Ryerson\n' 'Glencairn\n'
 'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale\n'
 'Rouge Hill, Port Union, Highland Creek\n' 'Woodbine Heights\n'
 'St. James Town\n' 'Humewood-Cedarvale\n'
 'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood\n'
 'Guildwood, Morningside, West Hill\n' 'The Beaches\n' 'Berczy Park\n'
 'Caledonia-Fairbanks\n' 'Woburn\n' 'Leaside\n' 'Central Bay Street\n'
 'Christie\n' 'Cedarbrae\n' 'Hillcrest Village\n'
 'Bathurst Manor, Wilson Heights, Downsview North\n' 'Thorncliffe Park\n'
 'Richmond, Adelaide, King\n' 'Dufferin, Dovercourt Village\n'
 'Scarborough Village\n' 'Fairview, Henry Farm, Oriole\n'
 'Northwood Park, York University\n'
 'East Toront

In [364]:
df.shape

(103, 3)

In [365]:
#import coordinates dataset in waston studio
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_1bab0aeef8cc4aba968a7303fbd8954c = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='cJCS782qjM895_8qG3MHemoSnT676gtkn7snEbZ8l2Q7',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_1bab0aeef8cc4aba968a7303fbd8954c.get_object(Bucket='capstoneproject-donotdelete-pr-4hgq4g1jminbx3',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [366]:
#merge two datasets
merged_left = pd.merge(df, df_data_1, on='Postal Code')
merged_left

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [368]:
#create map
address = 'Toronto'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [369]:
# plot map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_left['Latitude'], merged_left['Longitude'], merged_left['Borough'], merged_left['Neighborhood']):
    label = '{}, {}'.format(merged_left, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [370]:
#cluster only boroughs that contain the word Toronto
indexnames_1 = df[df['Borough'].str.contains('Toronto')].index
indexnames_1
merged_data = merged_left[ merged_left.index.isin(indexnames_1)].reset_index(drop=True)
merged_data['Borough'].unique()
#4 unique results

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

In [371]:
# one hot encoding
merge_onehot = pd.get_dummies(df['Borough'], prefix="", prefix_sep="")
merge_onehot['Neighborhood'] = df['Neighborhood'] 
merge_onehot['Postal Code'] = df['Postal Code'] 
merge_onehot
fixed_columns = [merge_onehot.columns[-1]] + list(merge_onehot.columns[:-1])
merge_onehot = merge_onehot[fixed_columns]
merge_onehot.head()

,Postal Code,Central Toronto,Downtown Toronto,East Toronto,East York,Etobicoke,Mississauga,North York,Scarborough,West Toronto,York,Neighborhood
0,M3A,0,0,0,0,0,0,1,0,0,0,Parkwoods
1,M4A,0,0,0,0,0,0,1,0,0,0,Victoria Village
2,M5A,0,1,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront"
3,M6A,0,0,0,0,0,0,1,0,0,0,"Lawrence Manor, Lawrence Heights"
4,M7A,0,1,0,0,0,0,0,0,0,0,"Queen's Park, Ontario Provincial Government"


In [372]:
merge_onehot.shape

(103, 12)

In [373]:
merge_grouped = merge_onehot.groupby('Neighborhood').mean().reset_index()
merge_grouped

,Neighborhood,Central Toronto,Downtown Toronto,East Toronto,East York,Etobicoke,Mississauga,North York,Scarborough,West Toronto,York
0,Agincourt,0,0,0,0,0,0,0,1,0,0
1,"Alderwood, Long Branch",0,0,0,0,1,0,0,0,0,0
2,"Bathurst Manor, Wilson Heights, Downsview North",0,0,0,0,0,0,1,0,0,0
3,Bayview Village,0,0,0,0,0,0,1,0,0,0
4,"Bedford Park, Lawrence Manor East",0,0,0,0,0,0,1,0,0,0
5,Berczy Park,0,1,0,0,0,0,0,0,0,0
6,"Birch Cliff, Cliffside West",0,0,0,0,0,0,0,1,0,0
7,"Brockton, Parkdale Village, Exhibition Place",0,0,0,0,0,0,0,0,1,0
8,"Business reply mail Processing Centre, South C...",0,0,1,0,0,0,0,0,0,0
9,"CN Tower, King and Spadina, Railway Lands, Har...",0,1,0,0,0,0,0,0,0,0


In [374]:
merge_grouped.shape

(99, 11)

In [375]:
num_top_venues = 5

for hood in merge_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = merge_grouped[merge_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))

----Agincourt----
              venue  freq
0       Scarborough   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3      East Toronto   0.0
4         East York   0.0
----Alderwood, Long Branch----
              venue  freq
0         Etobicoke   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3      East Toronto   0.0
4         East York   0.0
----Bathurst Manor, Wilson Heights, Downsview North----
              venue  freq
0        North York   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3      East Toronto   0.0
4         East York   0.0
----Bayview Village----
              venue  freq
0        North York   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3      East Toronto   0.0
4         East York   0.0
----Bedford Park, Lawrence Manor East----
              venue  freq
0        North York   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3      East Toronto   0.0
4         East York   0.0
----Berczy Park----
              venue  freq
0  

----Islington Avenue, Humber Valley Village----
              venue  freq
0         Etobicoke   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3      East Toronto   0.0
4         East York   0.0
----Kennedy Park, Ionview, East Birchmount Park----
              venue  freq
0       Scarborough   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3      East Toronto   0.0
4         East York   0.0
----Kensington Market, Chinatown, Grange Park----
              venue  freq
0  Downtown Toronto   1.0
1   Central Toronto   0.0
2      East Toronto   0.0
3         East York   0.0
4         Etobicoke   0.0
----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
              venue  freq
0         Etobicoke   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3      East Toronto   0.0
4         East York   0.0
----Lawrence Manor, Lawrence Heights----
              venue  freq
0        North York   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3

              venue  freq
0  Downtown Toronto   1.0
1   Central Toronto   0.0
2      East Toronto   0.0
3         East York   0.0
4         Etobicoke   0.0
----Upper Rouge----
              venue  freq
0       Scarborough   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3      East Toronto   0.0
4         East York   0.0
----Victoria Village----
              venue  freq
0        North York   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3      East Toronto   0.0
4         East York   0.0
----West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale----
              venue  freq
0         Etobicoke   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3      East Toronto   0.0
4         East York   0.0
----Westmount----
              venue  freq
0         Etobicoke   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3      East Toronto   0.0
4         East York   0.0
----Weston----
              venue  freq
0              York   1.0
1   Central

In [376]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [378]:
num_top_venues = 8

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = merge_grouped['Neighborhood']

for ind in np.arange(merge_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(merge_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,Agincourt,Scarborough,York,West Toronto,North York,Mississauga,Etobicoke,East York,East Toronto
1,"Alderwood, Long Branch",Etobicoke,York,West Toronto,Scarborough,North York,Mississauga,East York,East Toronto
2,"Bathurst Manor, Wilson Heights, Downsview North",North York,York,West Toronto,Scarborough,Mississauga,Etobicoke,East York,East Toronto
3,Bayview Village,North York,York,West Toronto,Scarborough,Mississauga,Etobicoke,East York,East Toronto
4,"Bedford Park, Lawrence Manor East",North York,York,West Toronto,Scarborough,Mississauga,Etobicoke,East York,East Toronto


In [379]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 4

merge_grouped_clustering = merge_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(merge_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 1, 1, 1, 3, 2, 0, 0, 3], dtype=int32)

In [382]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = merged_data

merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,East Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York


In [383]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [384]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
4,East Toronto,0,East Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
9,West Toronto,0,West Toronto,York,Scarborough,North York,Mississauga,Etobicoke,East York,East Toronto
11,West Toronto,0,West Toronto,York,Scarborough,North York,Mississauga,Etobicoke,East York,East Toronto
12,East Toronto,0,East Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
14,West Toronto,0,West Toronto,York,Scarborough,North York,Mississauga,Etobicoke,East York,East Toronto
15,East Toronto,0,East Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
17,East Toronto,0,East Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
18,Central Toronto,0,Central Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
19,Central Toronto,0,Central Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
20,Central Toronto,0,Central Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York


In [385]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue


In [386]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue


In [387]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
1,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
2,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
3,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
5,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
6,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
7,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
8,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
10,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
13,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
